In [1]:
import obd
import datetime
import csv
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


matplotlib.use("tkAgg")

#ser = serial.Serial('COM6', 9600)
#ser.flushInput()

plot_window = 20
rpm_var = np.array(np.zeros([plot_window]))
speed_var = np.array(np.zeros([plot_window]))
tp_var = np.array(np.zeros([plot_window]))

plt.ion()
fig, ax = plt.subplots()
ax.set_title('Drive Guard Data Collection')
ax.set_xlabel('Time')
ax.set_ylabel('Parameters')

rpmline, = ax.plot(rpm_var)
speedline, = ax.plot(speed_var)
tpline, = ax.plot(tp_var)
fields = ['Time','RPM','Speed','Throttle Position']
f = open(".\TrainingData.csv", "a+")
writer = csv.writer(f, delimiter=',')
writer.writerow(fields)


# Create an OBD connection using a COM port
connection = obd.OBD(portstr='COM3', baudrate=9600)

# Check if the connection was successfully established
if connection.is_connected():
    print("OBD connection successful")
else:
    print("Failed to connect to OBD")

while True:
    try:
        # Execute an OBD command
        cmd_rpm = obd.commands.RPM
        response_rpm = connection.query(cmd_rpm)
        print(f"Engine RPM: {response_rpm.value} {response_rpm.unit}")

        # Execute an OBD command
        cmd_speed = obd.commands.SPEED
        response_speed = connection.query(cmd_speed)
        print(f"Vehicle SPEED: {response_speed.value} {response_speed.unit}")

        # Execute an OBD command
        cmd_tp = obd.commands.THROTTLE_POS
        response_tp = connection.query(cmd_tp)
        print(f"Throttle Position: {response_tp.value} {response_tp.unit}")

        if response_rpm != "":
            rows = [response_rpm.value, response_speed.value, response_tp.value]
            print(rows)
            time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            rows.insert(0, time)
            print(rows)
            writer.writerow(rows)

            rpm_var = np.append(response_rpm.value, rows[1])
            rpm_var = rpm_var[1:plot_window + 1]
            rpmline.set_ydata(rpm_var)

            speed_var = np.append(response_speed.value, rows[2])
            speed_var = speed_var[1:plot_window + 1]
            speedline.set_ydata(speed_var)

            tp_var = np.append(response_tp.value, rows[3])
            tp_var = tp_var[1:plot_window + 1]
            tpline.set_ydata(tp_var)

            ax.relim()
            ax.autoscale_view()
            ax.legend([rpmline, speedline, tpline], ['RPM', 'Speed', 'Throttle Position'])
            fig.canvas.draw()
            fig.canvas.flush_events()

            f.flush()
    except:
        print("Keyboard Interrupt")
        # Close the connection
        connection.close()
        break

SyntaxError: invalid syntax (__init__.py, line 45)